In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [2]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print('Файл "{name}" завантажено. Розмір: {length} байт'.format(
        name=filename, length=len(uploaded[filename])))

Saving dataset_instagram.json to dataset_instagram (2).json
Файл "dataset_instagram (2).json" завантажено. Розмір: 18714695 байт


In [15]:
# import json

filename = 'dataset_instagram.json'
with open(filename, 'r') as file:
    data = json.load(file)

for i in range(5):
    print(data[i])

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 207: ordinal not in range(128)

In [17]:
#створення списку лише з постів
captions = []

for item in data:
    if 'caption' in item:
        captions.append(item['caption'])

# for caption in captions:
#     print(caption)

In [19]:
# заміна \n на пробіл у кожному елементі списку
captions = [elem.replace("\n", " ") for elem in captions]

# captions

In [6]:
# список у текстовий файл
with open("posts.txt", "w") as f:
    for post in captions:
        f.write(post + "\n")

In [7]:
# читання змісту файлу
# with open("posts.txt", "r") as file:
#     content = file.read()
#     # print(content)

In [8]:
# завантаження токенізатора та моделі
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

# завантаження датасету
dataset = TextDataset(tokenizer=tokenizer, file_path="posts.txt", block_size=128)

# підготовка датасету для тренування
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [9]:
# !pip install transformers==4.30
# !pip install accelerate>=0.21.0

In [10]:
# налаштування аргументів тренування
training_args = TrainingArguments(
    output_dir="./gpt2-medium-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    learning_rate=5e-5,
)

# створення об'єкта тренера
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# тренування
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.594000
1000,1.734000
1500,1.183300
2000,0.815300
2500,0.609500
3000,0.487400
3500,0.416300


TrainOutput(global_step=3760, training_loss=1.0695017307362658, metrics={'train_runtime': 3071.1524, 'train_samples_per_second': 9.775, 'train_steps_per_second': 1.224, 'total_flos': 6969898712432640.0, 'train_loss': 1.0695017307362658, 'epoch': 20.0})

In [ ]:
# збереження навченої моделі
trainer.save_model("./gpt2-instagram")

In [ ]:
from google.colab import files

files.download("./gpt2-instagram")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# функція для генерації тексту
def generate_text(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [12]:
# використання
prompt = "write me a post about the giveaway of 3 bags of the new collection"
generated_post = generate_text(prompt)
print("Згенерований пост:")
print(generated_post)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Згенерований пост:
write me a post about the giveaway of 3 bags of the new collection 🥤️  Enter via link in bio! (US Only)
"The soul always knows what it wants, and what it needs, before it can even think about going." - Alice Walker | 📷 by: @danholzphoto | Featured pack from the Seral/Savu Series #ospreypacks #thegooddaysaremade
"Travel sparks your imagination, feeds your curiosity, and carries


In [1]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Автентифікація та створення об'єкту GoogleDrive
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)


In [ ]:
# # Завантаження файлів на Google Диск
# model_file = drive.CreateFile({'title': 'gpt2-large-finetuned.pth'})
# model_file.SetContentFile('./gpt2-large-finetuned/pytorch_model.bin')
# model_file.Upload()
